In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import time

# Constants
EMBEDDING_DIM = 128
LSTM_UNITS = 128
USE_BIDIRECTIONAL = False
BATCH_SIZE = 128
EPOCHS = 50
TEMPERATURE = 1.0
MAX_GEN_LEN = 165
FP_RADIUS = 2
FP_NBITS = 2048
START_TOKEN = '!'
END_TOKEN = 'E'

# 1. Data Preparation
def prepare_data(file_path):
    # Load data from Excel file
    df = pd.read_excel(file_path)
    smiles_data = df['SMILES_1'].tolist()  # Assuming SMILES strings are in 'SMILES' column
    
    # Validate SMILES strings using RDKit
    valid_smiles = []
    for smile in smiles_data:
        mol = Chem.MolFromSmiles(smile)
        if mol is not None:
            valid_smiles.append(smile)
    
    print(f"Valid SMILES strings: {len(valid_smiles)} out of {len(smiles_data)}")
    
    # Preprocess SMILES strings (add start and end tokens)
    processed_smiles = [START_TOKEN + smile + END_TOKEN for smile in valid_smiles]
    
    # Create vocabulary
    unique_chars = set()
    for smile in processed_smiles:
        unique_chars.update(smile)
    
    vocab = sorted(list(unique_chars))
    vocab_size = len(vocab)
    
    print(f"Vocabulary size: {vocab_size}")
    
    # Create mapping dictionaries
    char_to_int = {char: i for i, char in enumerate(vocab)}
    int_to_char = {i: char for i, char in enumerate(vocab)}
    
    # Generate sequences for training
    X = []
    y = []
    
    for smile in processed_smiles:
        for i in range(1, len(smile)):
            X.append([char_to_int[char] for char in smile[:i]])
            y.append(char_to_int[smile[i]])
    
    print(f"Total sequences generated: {len(X)}")
    
    # Find maximum sequence length
    maxlen = max([len(seq) for seq in X]) + 1
    print(f"Maximum sequence length: {maxlen}")
    
    # Pad sequences
    X_padded = pad_sequences(X, maxlen=maxlen-1, padding='pre')
    
    # One-hot encode targets
    y_one_hot = np.zeros((len(y), vocab_size))
    for i, target in enumerate(y):
        y_one_hot[i, target] = 1
    
    # Create canonical SMILES set for novelty checking
    train_smiles_canonical_set = set()
    for smile in valid_smiles:
        mol = Chem.MolFromSmiles(smile)
        if mol is not None:
            canonical = Chem.MolToSmiles(mol)
            train_smiles_canonical_set.add(canonical)
    
    # Split data into training and validation sets (80/20)
    indices = np.arange(len(X_padded))
    np.random.shuffle(indices)
    
    train_idx = indices[:int(0.8 * len(indices))]
    val_idx = indices[int(0.8 * len(indices)):]
    
    X_train, y_train = X_padded[train_idx], y_one_hot[train_idx]
    X_val, y_val = X_padded[val_idx], y_one_hot[val_idx]
    
    print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")
    
    return X_train, y_train, X_val, y_val, char_to_int, int_to_char, vocab_size, maxlen, train_smiles_canonical_set

# 2. Model Building
def build_model(vocab_size, maxlen, embedding_dim=EMBEDDING_DIM, lstm_units=LSTM_UNITS, use_bidirectional=USE_BIDIRECTIONAL):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_shape=(maxlen-1,)))
    
    if use_bidirectional:
        from tensorflow.keras.layers import Bidirectional
        model.add(Bidirectional(LSTM(lstm_units)))
    else:
        model.add(LSTM(lstm_units))
    
    model.add(Dense(vocab_size, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    
    return model

# 3. Model Training
def train_model(model, X_train, y_train, X_val, y_val, batch_size=BATCH_SIZE, epochs=EPOCHS):
    # Define callbacks
    checkpoint = ModelCheckpoint('smiles_generator_model.keras', 
                                monitor='val_loss', 
                                save_best_only=True)
    
    early_stopping = EarlyStopping(monitor='val_loss', 
                                  patience=5, 
                                  restore_best_weights=True)
    
    # Train model
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(X_val, y_val),
                        callbacks=[checkpoint, early_stopping])
    
    return history, model

# 4. SMILES Generation
def generate_smiles(model, char_to_int, int_to_char, maxlen, temp=TEMPERATURE):
    # Start with the start token
    sequence = [char_to_int[START_TOKEN]]
    generated_smiles = START_TOKEN
    
    # Generate characters until we reach end token or max length
    while len(generated_smiles) < MAX_GEN_LEN:
        # Pad the sequence to the required input length
        padded_sequence = pad_sequences([sequence], maxlen=maxlen-1, padding='pre')[0]
        padded_sequence = padded_sequence.reshape(1, maxlen-1)
        
        # Predict the next character
        probs = model.predict(padded_sequence, verbose=0)[0]
        
        # Apply temperature scaling
        probs = np.log(probs) / temp
        exp_probs = np.exp(probs)
        probs = exp_probs / np.sum(exp_probs)
        
        # Sample the next character based on the predicted probabilities
        next_idx = np.random.choice(len(probs), p=probs)
        next_char = int_to_char[next_idx]
        
        # Append the character
        generated_smiles += next_char
        sequence.append(next_idx)
        
        # Check if we've reached the end token
        if next_char == END_TOKEN:
            break
    
    # Remove start and end tokens for the final SMILES
    final_smiles = generated_smiles[1:].replace(END_TOKEN, '')
    
    return final_smiles

# 5. Validation and Analysis
def generate_and_validate_molecules(model, char_to_int, int_to_char, maxlen, train_smiles_canonical_set, n_molecules=100):
    start_time = time.time()
    generated_molecules = []
    generated_canonical_set = set()  # For checking duplicates within this run
    
    stats = {
        'attempts': 0,
        'invalid': 0,
        'duplicate': 0,
        'known': 0,
        'accepted': 0
    }
    
    # Pre-compute fingerprints for training molecules for similarity calculation
    train_fps = []
    for canonical_smiles in train_smiles_canonical_set:
        mol = Chem.MolFromSmiles(canonical_smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, FP_RADIUS, nBits=FP_NBITS)
        train_fps.append(fp)
    
    while stats['accepted'] < n_molecules:
        stats['attempts'] += 1
        
        # Generate a new SMILES string
        generated_smiles = generate_smiles(model, char_to_int, int_to_char, maxlen, TEMPERATURE)
        
        # Check if it's a valid molecule
        mol = Chem.MolFromSmiles(generated_smiles)
        if mol is None:
            stats['invalid'] += 1
            continue
        
        # Convert to canonical SMILES
        canonical_smiles = Chem.MolToSmiles(mol)
        
        # Check if it's a duplicate within this run
        if canonical_smiles in generated_canonical_set:
            stats['duplicate'] += 1
            continue
        
        # Check if it's a known molecule (from training set)
        if canonical_smiles in train_smiles_canonical_set:
            stats['known'] += 1
            continue
        
        # If we reach here, the molecule is valid, unique, and novel
        # Calculate similarity to training set
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, FP_RADIUS, nBits=FP_NBITS)
        similarities = DataStructs.BulkTanimotoSimilarity(fp, train_fps)
        max_similarity = max(similarities)
        
        # Store the molecule information
        generated_molecules.append({
            'SMILES': generated_smiles,
            'Canonical_SMILES': canonical_smiles,
            'Novel': True,
            'Max_Similarity': max_similarity
        })
        
        # Add to set of generated molecules for duplicate checking
        generated_canonical_set.add(canonical_smiles)
        
        stats['accepted'] += 1
        
        if stats['accepted'] % 10 == 0:
            print(f"Generated {stats['accepted']} valid molecules in {stats['attempts']} attempts")
    
    # Create DataFrame and sort by similarity
    results_df = pd.DataFrame(generated_molecules)
    results_df = results_df.sort_values('Max_Similarity', ascending=True)
    
    elapsed_time = time.time() - start_time
    
    print(f"\nGeneration complete in {elapsed_time:.2f} seconds")
    print(f"Total attempts: {stats['attempts']}")
    print(f"Invalid SMILES: {stats['invalid']}")
    print(f"Duplicates: {stats['duplicate']}")
    print(f"Known molecules: {stats['known']}")
    print(f"Accepted novel molecules: {stats['accepted']}")
    
    # Save results
    results_df.to_csv('/kaggle/working/generated_molecules_analysis.csv', index=False)

    
    return results_df, stats

# Main execution flow
def main():
    file_path = '/kaggle/input/dataset20k/Data.xlsx'
    
    # 1. Prepare data
    X_train, y_train, X_val, y_val, char_to_int, int_to_char, vocab_size, maxlen, train_smiles_canonical_set = prepare_data(file_path)
    
    # 2. Build model
    model = build_model(vocab_size, maxlen)
    
    # 3. Train model
    history, trained_model = train_model(model, X_train, y_train, X_val, y_val)
    
    # 4 & 5. Generate and validate molecules
    results_df, stats = generate_and_validate_molecules(
        trained_model, char_to_int, int_to_char, maxlen, train_smiles_canonical_set
    )
    
    print("\nTop 10 most novel molecules (lowest similarity):")
    print(results_df.head(10)[['Canonical_SMILES', 'Max_Similarity']])
    print(results_df)

if __name__ == "__main__":
    main()

2025-04-17 17:56:15.889316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744912576.159059      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744912576.231287      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Valid SMILES strings: 20000 out of 20000
Vocabulary size: 20
Total sequences generated: 324314
Maximum sequence length: 29
Training samples: 259451, Validation samples: 64863


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1744912604.889708      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744912604.890498      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 28, 128)             │           2,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 20)                  │           2,580 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

...

[18:02:33] DEPRECATION WARNING: please use MorganGenerator
[18:02:34] DEPRECATION WARNING: please use MorganGenerator
[18:02:37] DEPRECATION WARNING: please use MorganGenerator
[18:02:38] DEPRECATION WARNING: please use MorganGenerator
[18:02:39] DEPRECATION WARNING: please use MorganGenerator
[18:02:43] DEPRECATION WARNING: please use MorganGenerator
[18:02:44] DEPRECATION WARNING: please use MorganGenerator
[18:02:45] DEPRECATION WARNING: please use MorganGenerator
[18:02:48] SMILES Parse Error: unclosed ring for input: 'CC12OC3C4C5C1C2N34'
[18:02:51] Explicit valence for atom # 8 F, 3, is greater than permitted
[18:02:52] DEPRECATION WARNING: please use MorganGenerator
[18:02:53] DEPRECATION WARNING: please use MorganGenerator


Generated 90 valid molecules in 118 attempts


[18:02:54] DEPRECATION WARNING: please use MorganGenerator
[18:02:55] SMILES Parse Error: unclosed ring for input: 'OCC12COC(C1)C1O2'
[18:02:56] DEPRECATION WARNING: please use MorganGenerator
[18:02:57] DEPRECATION WARNING: please use MorganGenerator
[18:02:58] DEPRECATION WARNING: please use MorganGenerator
[18:02:59] DEPRECATION WARNING: please use MorganGenerator
[18:03:00] DEPRECATION WARNING: please use MorganGenerator
[18:03:01] DEPRECATION WARNING: please use MorganGenerator
[18:03:02] SMILES Parse Error: unclosed ring for input: 'O=C1C2C3C2C3OC13'
[18:03:03] DEPRECATION WARNING: please use MorganGenerator
[18:03:05] DEPRECATION WARNING: please use MorganGenerator


Generated 100 valid molecules in 131 attempts

Generation complete in 138.58 seconds
Total attempts: 131
Invalid SMILES: 8
Duplicates: 0
Known molecules: 23
Accepted novel molecules: 100

Top 10 most novel molecules (lowest similarity):
              Canonical_SMILES  Max_Similarity
28  [NH3+]CC1N=C([O-])C=[NH+]1        0.228571
73         C#CN(C)N=C1CC(=O)C1        0.277778
5            CCC1=NC(C2CC2)=C1        0.290323
56           CC(=O)C1CCC(O)=N1        0.300000
9              CC1=CC2OC1OC2=N        0.322581
19            CC1CN1C(C)(C)C#N        0.333333
61                CCCn1ccn1C=O        0.343750
6              CCN1CC(C)(O)C1O        0.379310
23             CCC12CC(C1)OC2O        0.387097
99             CN=C1OCC(=O)N1C        0.400000
                        SMILES            Canonical_SMILES  Novel  \
28  [NH3+]CC1[NH+]=CC([O-])=N1  [NH3+]CC1N=C([O-])C=[NH+]1   True   
73         CN(N=C1CC(=O)C1)C#C         C#CN(C)N=C1CC(=O)C1   True   
5            CCC1=NC(=C1)C1CC1         

[18:03:06] DEPRECATION WARNING: please use MorganGenerator


In [3]:
from IPython.display import FileLink

# Make a clickable link
FileLink('generated_molecules_analysis.csv')


/kaggle/working/generated_molecules_analysis.csv

In [1]:
pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 49.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.
